In [1]:
# 어텐션 메커니즘
# 가중치 계산
# query vector, keys vectors 유사도 기반 내적 계산 >> attention score
# sfotmax 함수 이용, 가중치 계산
# keys vector에 가중평균 적용 >> context vector 생성
# query 와 key 를 입력으로 이용

In [2]:
import numpy as np

# 나는 집에 -> 갑니다 (2차원 벡터) >> 1차원 벡터
# 입력문장에 대한 hidden state : keys

keys = np.array([[0.11, 0.12, 0.13], [0.21, 0.22, 0.23]])  # 나는, 집에


# 현재 출력 시점에서의 hidden state # query

query = np.array([0.41, 0.42, 0.43])  # 갑니다

print(keys.shape, query.shape)

(2, 3) (3,)


In [3]:
# 어텐션 메커니즘

# 1. attention score 계산(유사도 반영)

scores = np.dot(keys, query)

print(scores.shape)

(2,)


In [5]:
# 2. Attention weights 계산(softmax) >> 중요도 계산

weights = np.exp(scores) / np.sum(np.exp(scores))
weights

array([0.46854161, 0.53145839])

In [8]:
weights_repeated = weights.repeat(3).reshape(2,3)
weights_repeated

array([[0.46854161, 0.46854161, 0.46854161],
       [0.53145839, 0.53145839, 0.53145839]])

In [11]:
# 가중평균 계산 
context_vector = keys * weights_repeated
print('context_vector :\n', context_vector)

context_vector :
 [[0.05153958 0.05622499 0.06091041]
 [0.11160626 0.11692085 0.12223543]]


In [12]:
context_vector.sum(axis=0)

array([0.16314584, 0.17314584, 0.18314584])

In [14]:
context_vector = context_vector.sum(axis=0)

In [16]:
context_vector

array([0.16314584, 0.17314584, 0.18314584])

In [17]:
# 셀프 어텐션(Self-Attention)

# 하나의 문장 또는 문서에서 단어 간 상호작용을 모델링 하기 위해 사용하는 어텐션 메커니즘

# 셀프 어텐션 구현 단계
# 1. 임베딩 층을 통해 입력 문장을 임베딩 벡터로 변환
# 2. 입력 임베딩 벡터 >> query, key, value 구분해서 나눔 >>
# 3. 각각을 query, keys, values 변수에 저장
# 4. query와 key 내적, 어텐션 스코어(attention score) 계산
# 5. 어텐션 스코어에 softmax 함수 적용 >> (전체의 합이 1인 확률값 변환) >> 가중치(weights) 계산
# 6. 가중치(weights)와 value 곱하여 컨텍스트 벡터 (context vector) 계산

In [41]:
import numpy as np

def self_attention(inputs) :
    # inputs : [batch_size, seq_len, d_model]
    
    # query, key, value 생성
    # 실제 q, k, v 입력값은 다름(편의상 같은 값 입력값으로 사용)
    
    q = inputs
    k = inputs
    v = inputs
    
    # 1. 행렬 곱(내적) >> attention score 계산
    scores = np.matmul(q, k.transpose(0, 2, 1))
    
    # 2. 스케일링(scaled)작업
    d_k = q.shape[-1]
    scores /= np.sqrt(d_k)
    
    # 3. softmax 함수 적용, 가중치 계산
    weights = np.exp(scores) / np.sum(np.exp(scores), axis = -1, keepdims = True)
    
    # 4. 가중치(weights)와 values 를 곱함(내적) >>  context vector 계산
    context_vector = np.matmul(weights, v)
    
    return context_vector

In [42]:
# 입력문장에 대한 hidden state : keys 

inputs = np.array([[[0.11, 0.12, 0.13],  # 나는 
                    [0.21, 0.22, 0.23],  # 집에
                    [0.31, 0.32, 0.33]]]) # 갑니다

In [43]:
inputs.shape

(1, 3, 3)

In [44]:
inputs.transpose(0,2,1), inputs.transpose(0,2,1).shape

(array([[[0.11, 0.21, 0.31],
         [0.12, 0.22, 0.32],
         [0.13, 0.23, 0.33]]]),
 (1, 3, 3))

In [53]:
inputs.transpose(), inputs.transpose().shape, inputs.transpose(2,1,0)

(array([[[0.11],
         [0.21],
         [0.31]],
 
        [[0.12],
         [0.22],
         [0.32]],
 
        [[0.13],
         [0.23],
         [0.33]]]),
 (3, 3, 1),
 array([[[0.11],
         [0.21],
         [0.31]],
 
        [[0.12],
         [0.22],
         [0.32]],
 
        [[0.13],
         [0.23],
         [0.33]]]))

In [47]:
context_vector = self_attention(inputs)
print('context_vector :\n', context_vector)

context_vector :
 [[[0.21138554 0.22138554 0.23138554]
  [0.21253973 0.22253973 0.23253973]
  [0.21369315 0.22369315 0.23369315]]]


In [56]:
# Multi-Head attention

import tensorflow as tf

# 입력문장을 임베딩 한 후, 셀프 어텐션을 수행하는 함수

def multi_head_attention(embeddings) :
    # 셀프 어텐션 수행
    attention_output = tf.keras.layers.MultiHeadAttention(
                       num_heads = 8, key_dim = 512, 
                       )(embeddings, embeddings)
    
    # 셀프어텐션 출력값 >> 임베딩벡터에 더함 >> 최종 출력값 생성
    add_output = tf.keras.layers.Add()([attention_output, embeddings])
    
    # 출력값을 정규화
    normalization_output = tf.keras.layers.LayerNormalization()(add_output)
    
    return normalization_output

In [57]:
# 예시 입력 문장
inputs = tf.keras.layers.Input(shape = (10, 512))

# 입력 임베딩
embeddings = tf.keras.layers.Embedding(input_dim = 10000, output_dim = 512)(inputs)

# 멀티 헤더 어텐션 수행
attention_output = multi_head_attention(embeddings)
attention_output

<KerasTensor: shape=(None, 10, 512, 512) dtype=float32 (created by layer 'layer_normalization')>